<a href="https://colab.research.google.com/github/ChennyTech/TensorflowStudyExample/blob/master/pytorch_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms


# ================================================================== #
#                         Table of Contents                          #
# ================================================================== #

# 1. Basic autograd example 1               (Line 25 to 39)
# 2. Basic autograd example 2               (Line 46 to 83)
# 3. Loading data from numpy                (Line 90 to 97)
# 4. Input pipline                          (Line 104 to 129)
# 5. Input pipline for custom dataset       (Line 136 to 156)
# 6. Pretrained model                       (Line 163 to 176)
# 7. Save and load model                    (Line 183 to 189) 


# ================================================================== #
#                     1. Basic autograd example 1                    #
# ================================================================== #

# Create tensors.
x = torch.tensor(1., requires_grad=True) # 첫 argument에 matrix 넣어주기 [] 이용하여
w = torch.tensor(2., requires_grad=True) # false로 하면 grad에 none이 나옴
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
y.backward() # 위 식에서 변수 하나하나에 대해 미분


# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

None
tensor(1.)
tensor(1.)


In [13]:
# ================================================================== #
#                    2. Basic autograd example 2                     #
# ================================================================== #

# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3) # m 0 sigma^2 1의 Gaussian normal distribution 따르는 10 X 5 tensor 생성
y = torch.randn(10, 2) # rand 는 0~1 균일 분포 normal distribution, randint는 random scalar


# Build a fully connected layer.
linear = nn.Linear(3, 2)

# nn.Linear은 nn.Module의 subclass로
# in_features개의 input을 선형변환을 거쳐
# out_features개의 output으로 변환

# initialization?
print ('w: ', linear.weight)
print ('b: ', linear.bias)

# Build loss function and optimizer.
criterion = nn.MSELoss() # min square error
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)
print(pred)

# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item()) # scalar값

# Backward pass.
loss.backward() # Loss summation 식 

# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) # weight 하나하나에 대해 미분 -> w size로 나옴
print ('dL/db: ', linear.bias.grad) # same


# 1-step gradient descent.
optimizer.step() # optimizer 지정해서 update

# You can also perform gradient descent at the low level.
linear.weight.data.sub_(0.01 * linear.weight.grad.data) # 전달 인자를 subtraction
linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())




w:  Parameter containing:
tensor([[-0.2148, -0.5641,  0.2255],
        [-0.3099, -0.0164,  0.0307]], requires_grad=True)
b:  Parameter containing:
tensor([-0.3232,  0.1939], requires_grad=True)
tensor([[ 0.3936,  0.0432],
        [ 0.7058, -0.2333],
        [ 0.0595,  0.1623],
        [-0.0866,  0.6513],
        [-0.3590,  0.5420],
        [ 0.1163,  0.8969],
        [-0.1574,  0.1974],
        [-0.3340, -0.1954],
        [-0.2657,  0.0461],
        [-1.6222, -0.5753]], grad_fn=<AddmmBackward>)
loss:  2.193894624710083
dL/dw:  tensor([[-1.1609, -0.5982, -0.7868],
        [-0.5729,  0.3639, -0.1143]])
dL/db:  tensor([-0.6714,  0.7317])
loss after 1 step optimization:  2.1195459365844727


In [0]:
# ================================================================== #
#                     3. Loading data from numpy                     #
# ================================================================== #

# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

In [20]:
# ================================================================== #
#                         4. Input pipline                           #
# ================================================================== #

# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0] # 데이터 읽어오기 
print (image.size()) # 3 X 32 X 32
print (label) # category

# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True) # 64 samples 씩, shuffle data at every epoch

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next() # next train_dataset 으로
# image는 3 X 32 X 32 가 64개 있고, label은 scalar가 64개

# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass




0it [00:00, ?it/s]

170500096it [00:04, 42391747.32it/s]                               


Extracting ../../data/cifar-10-python.tar.gz to ../../data/
torch.Size([3, 32, 32])
6


In [0]:
# ================================================================== #
#                5. Input pipline for custom dataset                 #
# ================================================================== #

# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 100 

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)



torch.Size([64, 100])
torch.Size([64, 3, 224, 224])


<All keys matched successfully>

In [21]:
# ================================================================== #
#                        6. Pretrained model                         #
# ================================================================== #

# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False # gradient 계산 안하겠다는 것

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example. # output feature가 100

# Forward pass.
images = torch.randn(64, 3, 224, 224) # 3개의 224 X 224 가 -> 64개 input features
outputs = resnet(images)
print (outputs.size())     # (64, 100) # 마지막 layer output feature를 100으로 설정했으니까


# ================================================================== #
#                      7. Save and load the model                    #
# ================================================================== #

# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt') # weight와 bias 저장
resnet.load_state_dict(torch.load('params.ckpt'))

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 68.1MB/s]


torch.Size([64, 100])


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist

<All keys matched successfully>